In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
bigdata = requests.get('https://arxiv.org/html/2401.00009v1')
soup = BeautifulSoup(bigdata.text, "lxml")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get('https://arxiv.org/html/2401.00009v1', headers=headers)

abstract = soup.findAll("div",{"class":"ltx_abstract"})
keyword = soup.findAll("div",{"class":"ltx_keywords"})
author = soup.findAll("span",{"class":"ltx_contact ltx_role_affiliation"})

# data = []

# abstract_text = " ".join([item.get_text() for item in abstract])
# keyword_text = " ".join([item.get_text().replace("Keywords: ", "").strip() for item in keyword])
# keywords_array = re.split(r'(?<!^)\s(?=[A-Z])', keyword_text)
# author_text = " ".join([item.get_text() for item in author])

# countries = [item.get_text().strip().split()[-1] for item in author]

# data.append({'abstract':abstract_text, 'keyword':keywords_array, 'country':countries})

# data

# if "reCAPTCHA" in soup.get_text():
#     print("Blocked by CAPTCHA. Exiting...")

soup.get_text()

'\n\n\nTuring’s Test, a Beautiful Thought Experiment\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 Introduction\n2 What is the Turing Test?\n3 Imitation: from 1936 to 1950\n4 The method of thought experiments\n5 1949, the crucial year\n6 A hammer and a feather on the moon\n7 The value of Turing’s test for AI\n8 Conclusion\n\n\n\nLicense: CC BY 4.0arXiv:2401.00009v1 [cs.AI] 18 Dec 2023\n\nTuring’s Test, a Beautiful Thought Experiment\n\n\nBernardo Gonçalves\u2006\n\n0000-0003-2794-8478\nPolytechnic School, University of São PauloBrazil\n\nbegoncalves@usp.br\nAlso Visiting Fellow at King’s College and Research Affiliate at the Department of History and Philosophy of Science, University of CambridgeUK\n\n(2024)\n\nAbstract.\nIn the wake of large language models, there has been a resurgence of claims and questions about the Turing test and its value for AI, which are reminiscent of decades of practical “Turing” tests. If AI were quantum physics, by now several “Schrödinger’s” cats could have been kille

In [15]:
data = []

# Start at January 2024 (2401)
month = 1
year = 24
paper_id = 1  # Global paper ID
not_found_count = 0

while len(data) < 1000:
    # Format YYMM (e.g., "2401" for Jan 2024)
    yymm = f"{year:02}{month:02}"

    # Build URL with global paper_id
    url = f'https://arxiv.org/html/{yymm}.{paper_id:05}v1'
    print(f"The current paper id is {yymm}.{paper_id:05}")
    try:
        bigdata = requests.get(url)
        soup = BeautifulSoup(bigdata.text, "lxml")

        # Check for "not found" page
        if soup.find("h1", string=re.compile(r"Article .* not found")):
            not_found_count += 1
            if not_found_count == 2:  # If two consecutive "not found," no more papers exist
                break
            paper_id += 1  # Increment paper_id globally
            continue

        # Reset not_found_count when a valid page is found
        not_found_count = 0

        # Extract fields
        abstract = soup.select('div div .ltx_abstract .ltx_p')
        keyword = soup.select('div div .ltx_keywords')
        author = soup.findAll("span", {"class": "ltx_contact ltx_role_affiliation"})

        # Process data
        abstract_text = " ".join([item.get_text() for item in abstract]).strip()
        keyword_text = " ".join([item.get_text().replace("Keywords: ", "").strip() for item in keyword]).strip()
        keywords_array = re.split(r'(?<!^)\s(?=[A-Z])', keyword_text)
        author_text = " ".join([item.get_text() for item in author]).strip()
        countries = [item.get_text().strip().split()[-1] for item in author]

        # Skip if any key field is empty
        if abstract_text and keyword_text and author_text:
            data.append({'abstract': abstract_text, 'keyword': keywords_array, 'country': countries})

        # Break if we reach 1000 entries
        if len(data) >= 1000:
            break

        paper_id += 1  # Increment paper_id globally
    except Exception as e:
        print(f"Error on paper {yymm}.{paper_id:05}: {e}")
        paper_id += 1  # Ensure paper_id increments even on error

    # Move to the next month if two consecutive "not found" occur and paper_id crosses into a new month
    if not_found_count == 2:
        month += 1
        not_found_count = 0
        if month > 12:  # If December is exceeded, move to next year
            month = 1
            year += 1
    
    print(f"Scraped {len(data)} entries so far...")

    time.sleep(1)

print(f"Data collected: {len(data)} entries")


The current paper id is 2401.00001
Scraped 0 entries so far...
The current paper id is 2401.00002
Scraped 0 entries so far...
The current paper id is 2401.00003
Scraped 0 entries so far...
The current paper id is 2401.00004
Scraped 0 entries so far...
The current paper id is 2401.00005
Scraped 0 entries so far...
The current paper id is 2401.00006
Scraped 0 entries so far...
The current paper id is 2401.00007
Scraped 0 entries so far...
The current paper id is 2401.00008
Scraped 0 entries so far...
The current paper id is 2401.00009
Scraped 0 entries so far...
The current paper id is 2401.00010
Scraped 0 entries so far...
The current paper id is 2401.00011
Scraped 0 entries so far...
The current paper id is 2401.00012
Scraped 0 entries so far...
The current paper id is 2401.00013
Scraped 0 entries so far...
The current paper id is 2401.00014
Scraped 0 entries so far...
The current paper id is 2401.00015
Scraped 0 entries so far...
The current paper id is 2401.00016
Scraped 0 entries so

KeyboardInterrupt: 

In [12]:
df = pd.DataFrame(data)
df.to_csv('out2.csv', index=False)
df

,abstract,keyword,country
0,,[],[]
1,,[],[]
2,,[],[]
3,,[],[]
4,,[],[]
...,...,...,...
995,,[],[]
996,,[],[]
997,,[],[]
998,,[],[]
